In [2]:
import requests
import defs
import pandas as pd

In [3]:
session = requests.Session()

In [4]:
ins_df = pd.read_pickle('instruments.pkl')

In [5]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [6]:
ins_df.shape[0]

123

In [7]:
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         123 non-null    object
 1   type         123 non-null    object
 2   displayName  123 non-null    object
 3   pipLocation  123 non-null    int64 
 4   marginRate   123 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.9+ KB


In [8]:
ins_df.columns

Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [9]:
ins_df.name.unique()

array(['USD_HKD', 'USD_CNH', 'SGD_CHF', 'NZD_CAD', 'NZD_JPY', 'EUR_GBP',
       'US30_USD', 'JP225Y_JPY', 'EUR_NOK', 'XAU_USD', 'CN50_USD',
       'USD_CZK', 'XAU_NZD', 'CH20_CHF', 'CAD_HKD', 'XAG_CHF', 'USD_CHF',
       'AUD_USD', 'XAG_HKD', 'AUD_HKD', 'ESPIX_EUR', 'NZD_CHF', 'AUD_CHF',
       'GBP_CHF', 'USD_THB', 'XAU_JPY', 'XAU_HKD', 'EUR_HKD', 'CHF_JPY',
       'GBP_HKD', 'EUR_NZD', 'XAG_AUD', 'WTICO_USD', 'XAG_NZD', 'AUD_SGD',
       'EUR_JPY', 'EUR_TRY', 'USD_JPY', 'SGD_JPY', 'GBP_ZAR', 'XAG_JPY',
       'ZAR_JPY', 'NZD_SGD', 'CAD_JPY', 'USD_CAD', 'EUR_AUD', 'XAG_GBP',
       'EUR_SGD', 'CAD_CHF', 'BCO_USD', 'XAU_SGD', 'SPX500_USD',
       'XAU_XAG', 'USB10Y_USD', 'EU50_EUR', 'USD_NOK', 'USB02Y_USD',
       'HK33_HKD', 'XCU_USD', 'XAG_EUR', 'AUD_JPY', 'EUR_ZAR', 'NZD_USD',
       'XAU_GBP', 'GBP_AUD', 'USD_PLN', 'US2000_USD', 'HKD_JPY',
       'TRY_JPY', 'NATGAS_USD', 'USD_HUF', 'NAS100_USD', 'EUR_CAD',
       'USD_ZAR', 'DE30_EUR', 'XPT_USD', 'CORN_USD', 'XAU_EUR',
       'SOYB

In [12]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = 'MBA'
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [18]:
def get_candles_df(json_response):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f'{price}_{oh}'] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)



In [19]:
code, res = fetch_candles('EUR_USD', 10, 'H1')
df = get_candles_df(res)
df.head()


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2025-04-29T13:00:00.000000000Z,15299,1.14107,1.14174,1.13900,1.13980,1.14099,1.14167,1.13893,1.13964,1.14115,1.14182,1.13908,1.13997
1,2025-04-29T14:00:00.000000000Z,18273,1.13982,1.14178,1.13789,1.13832,1.13965,1.14169,1.13781,1.13825,1.13999,1.14186,1.13797,1.13839
2,2025-04-29T15:00:00.000000000Z,10849,1.13832,1.14047,1.13823,1.13960,1.13825,1.14040,1.13815,1.13952,1.13840,1.14055,1.13830,1.13968
3,2025-04-29T16:00:00.000000000Z,8823,1.13961,1.14190,1.13953,1.14035,1.13954,1.14182,1.13946,1.14028,1.13968,1.14197,1.13960,1.14042
4,2025-04-29T17:00:00.000000000Z,6409,1.14036,1.14048,1.13836,1.13860,1.14028,1.14041,1.13828,1.13851,1.14043,1.14056,1.13844,1.13868


In [ ]:
def save_file(candles_df, pair, granularity):    
    file_name = f"his_data/{pair}_{granularity}.pkl"
    candles_df.to_pickle(file_name)
    #print(f"Saved {file_name} with {candles_df.shape[0]} rows")

In [21]:
def create_data(pair, granularity):
    code, res = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(f"Error fetching {pair} data")
        return
    df = get_candles_df(res)
    if df.shape[0] == 0:
        print(f"No data for {pair}")
        return
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [ ]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, 'H1')